# Missing Value

In [37]:
using DataFrames, CSV, JuMP, Gurobi, Metrics, Plots, StatsBase

In [38]:
# load data
ratings_20 = CSV.read("../data/Final/ratings/ratings_20.csv", DataFrame)
ratings_21 = CSV.read("../data/Final/ratings/ratings_21.csv", DataFrame)
ratings_22 = CSV.read("../data/Final/ratings/ratings_22.csv", DataFrame);

In [29]:
# use IAI.ImputationLearner to impute missing values
lnr = IAI.ImputationLearner(method = :opt_tree, random_seed = 1234)

# separate numeric and string columns
string_columns = ["short_name", "club_name"]
numeric_columns = setdiff(names(ratings_20), string_columns);

# keep only numeric columns
ratings_20_numeric = ratings_20[:, numeric_columns]
ratings_21_numeric = ratings_21[:, numeric_columns]
ratings_22_numeric = ratings_22[:, numeric_columns];

# impute missing values
ratings_20_imputed = IAI.fit_transform!(lnr, ratings_20_numeric)


Row,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_level,club_joined,club_contract_valid_until,preferred_foot,weak_foot,skill_moves,international_reputation,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,position_A,position_M,position_D,position_G,attack_work_rate,defense_work_rate,year
,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?
1,94.0,94.0,9.55e7,560000.0,32.0,170.0,72.0,1.0,2004.0,2021.0,0.0,4.0,4.0,5.0,1.958e8,87.0,92.0,92.0,96.0,39.0,66.0,88.0,95.0,70.0,92.0,88.0,97.0,93.0,94.0,92.0,96.0,91.0,84.0,93.0,95.0,95.0,86.0,68.0,75.0,68.0,94.0,48.0,40.0,94.0,94.0,75.0,96.0,33.0,37.0,26.0,6.0,11.0,15.0,14.0,8.0,1.0,1.0,0.0,0.0,2.0,1.0,2020.0
2,93.0,93.0,5.85e7,410000.0,34.0,187.0,83.0,1.0,2004.0,2022.0,1.0,4.0,5.0,5.0,9.65e7,90.0,93.0,82.0,89.0,35.0,78.0,84.0,94.0,89.0,83.0,87.0,89.0,81.0,76.0,77.0,92.0,89.0,91.0,87.0,96.0,71.0,95.0,95.0,85.0,78.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,32.0,24.0,7.0,11.0,15.0,14.0,11.0,1.0,0.0,0.0,0.0,3.0,1.0,2020.0
3,92.0,92.0,1.055e8,290000.0,27.0,175.0,68.0,1.0,2004.0,2022.0,1.0,5.0,5.0,5.0,1.952e8,91.0,85.0,87.0,95.0,32.0,58.0,87.0,87.0,62.0,87.0,87.0,96.0,88.0,87.0,81.0,95.0,94.0,89.0,96.0,92.0,84.0,80.0,61.0,81.0,49.0,84.0,51.0,36.0,87.0,90.0,90.0,94.0,27.0,26.0,29.0,9.0,9.0,15.0,15.0,11.0,1.0,1.0,0.0,0.0,3.0,2.0,2020.0
4,91.0,91.0,9.0e7,470000.0,28.0,175.0,74.0,1.0,2004.0,2024.0,1.0,4.0,4.0,4.0,1.845e8,91.0,83.0,86.0,94.0,35.0,66.0,81.0,84.0,61.0,89.0,83.0,95.0,83.0,79.0,83.0,94.0,94.0,88.0,95.0,90.0,94.0,82.0,56.0,84.0,63.0,80.0,54.0,41.0,87.0,89.0,88.0,91.0,34.0,27.0,22.0,11.0,12.0,6.0,8.0,8.0,1.0,1.0,0.0,0.0,3.0,2.0,2020.0
5,91.0,91.0,9.0e7,370000.0,28.0,181.0,70.0,1.0,2004.0,2023.0,1.0,5.0,4.0,4.0,1.665e8,76.0,86.0,92.0,86.0,61.0,78.0,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,77.0,76.0,78.0,91.0,76.0,91.0,63.0,89.0,74.0,90.0,76.0,61.0,88.0,94.0,79.0,91.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,0.0,1.0,0.0,0.0,3.0,3.0,2020.0
6,91.0,93.0,7.75e7,125000.0,26.0,188.0,87.0,1.0,2004.0,2023.0,1.0,3.0,1.0,3.0,1.647e8,76.1482,54.5676,67.1261,64.9967,57.6492,74.3026,13.0,11.0,15.0,43.0,13.0,12.0,13.0,14.0,40.0,30.0,43.0,60.0,67.0,88.0,49.0,59.0,78.0,41.0,78.0,12.0,34.0,19.0,11.0,65.0,11.0,68.0,27.0,12.0,18.0,87.0,92.0,78.0,90.0,89.0,0.0,0.0,0.0,1.0,2.0,2.0,2020.0
7,90.0,90.0,4.5e7,340000.0,33.0,172.0,66.0,1.0,2004.0,2020.0,1.0,4.0,4.0,4.0,9.23e7,74.0,76.0,89.0,89.0,72.0,66.0,86.0,72.0,55.0,92.0,76.0,87.0,85.0,78.0,88.0,92.0,77.0,71.0,92.0,89.0,93.0,79.0,68.0,85.0,58.0,82.0,62.0,82.0,79.0,91.0,82.0,92.0,68.0,76.0,71.0,13.0,9.0,7.0,14.0,9.0,0.0,1.0,0.0,0.0,3.0,3.0,2020.0
8,90.0,93.0,6.75e7,250000.0,27.0,187.0,85.0,1.0,2004.0,2022.0,1.0,4.0,1.0,3.0,1.434e8,68.8236,58.2357,78.8075,66.792,56.7096,75.1626,18.0,14.0,11.0,61.0,14.0,21.0,18.0,12.0,63.0,30.0,38.0,50.0,37.0,86.0,43.0,66.0,79.

In [40]:
# just transform the two other datasets
ratings_21_imputed = IAI.transform(lnr, ratings_21_numeric)
ratings_22_imputed = IAI.transform(lnr, ratings_22_numeric);

In [30]:
# print goalkeepers
ratings_20_imputed[ratings_20_imputed.position_G .== 1, ["passing", "physic", "defending", "dribbling", "shooting", "pace"]]

Row,passing,physic,defending,dribbling,shooting,pace
,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?
1,67.1261,74.3026,57.6492,64.9967,54.5676,76.1482
2,78.8075,75.1626,56.7096,66.792,58.2357,68.8236
3,72.2356,66.5072,61.3393,69.5085,56.881,81.6056
4,69.7848,69.6657,53.7644,69.4199,58.2992,75.0253
5,58.1261,68.5899,55.6137,57.955,54.2667,76.4027
6,75.2996,74.9432,56.3483,75.5451,59.7514,80.9562
7,63.7593,56.2412,58.5178,63.6042,53.311,87.0619
8,59.5406,67.1303,55.2732,62.479,56.3556,73.3118
9,77.291,70.2133,58.9767,72.6244,60.392,87.7745


In [27]:
# print other players
ratings_20_imputed[ratings_20_imputed.position_G .== 0, ["passing", "physic", "defending", "dribbling", "shooting", "pace"]]

Row,passing,physic,defending,dribbling,shooting,pace
,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?
1,92.0,66.0,39.0,96.0,92.0,87.0
2,82.0,78.0,35.0,89.0,93.0,90.0
3,87.0,58.0,32.0,95.0,85.0,91.0
4,86.0,66.0,35.0,94.0,83.0,91.0
5,92.0,78.0,61.0,86.0,86.0,76.0
6,89.0,66.0,72.0,89.0,76.0,74.0
7,70.0,86.0,90.0,71.0,60.0,77.0
8,81.0,74.0,45.0,89.0,86.0,93.0
9,58.0,82.0,90.0,60.0,46.0,68.0


In [41]:
# add the string columns back
ratings_20_full = hcat(ratings_20[:, string_columns], ratings_20_imputed)
ratings_21_full = hcat(ratings_21[:, string_columns], ratings_21_imputed)
ratings_22_full = hcat(ratings_22[:, string_columns], ratings_22_imputed);

In [44]:
# print goalkeepers
ratings_22_full[ratings_22_full.position_G .== 1, ["short_name", "club_name", "overall", "passing", "physic", "defending", "dribbling", "shooting", "pace"]]

Row,short_name,club_name,overall,passing,physic,defending,dribbling,shooting,pace
,String31,String,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?
1,J. Oblak,Atlético de Madrid,91.0,67.9747,74.4229,57.6166,65.2937,54.9455,76.0214
2,M. Neuer,FC Bayern München,90.0,78.4351,74.3016,56.3159,75.4953,59.9996,80.6037
3,M. ter Stegen,FC Barcelona,90.0,79.3479,75.3436,56.7615,67.365,58.4457,69.3594
4,T. Courtois,Real Madrid CF,89.0,60.5359,67.7773,55.6508,63.2201,57.2334,72.6664
5,Ederson,Manchester City,89.0,80.4619,75.0205,58.9478,72.7776,61.9223,87.723
6,Alisson,Liverpool,89.0,70.5338,70.2121,54.8973,69.9353,58.3652,75.9564
7,G. Donnarumma,Paris Saint-Germain,89.0,63.1662,69.1641,57.3173,72.7455,57.1543,77.5313
8,K. Navas,Paris Saint-Germain,88.0,66.2372,71.4909,57.923,70.7759,56.9326,77.276
9,H. Lloris,Tottenham Hotspur,87.0,68.5014,56.565,58.7907,64.0163,53.231,86.1459


In [45]:
# save the data
CSV.write("../data/Final/ratings/ratings_20_imputed.csv", ratings_20_full)
CSV.write("../data/Final/ratings/ratings_21_imputed.csv", ratings_21_full)
CSV.write("../data/Final/ratings/ratings_22_imputed.csv", ratings_22_full);